In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet152 as Model
from torchvision.models.resnet import ResNet152_Weights as ModelW

In [2]:
input_shape = (3, 256, 256)
output_shape = 4

def cnn_final_convolution_size(initial_size, kernel_size, pooling_size):
    size = initial_size
    size = size - kernel_size + 1
    size = size // pooling_size
    size = size - kernel_size + 1
    size = size // pooling_size
    return size

Basic Model

In [3]:
class MRI_CNN(nn.Module):
    def __init__(self, kernel_size=5, conv_out_features_1=32, conv_out_features_2=64, pooling_size=2, linear_size_1=512, linear_size_2=128):
        super(MRI_CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_shape[0], conv_out_features_1, kernel_size)
        self.pool = nn.MaxPool2d(pooling_size, pooling_size)
        self.conv2 = nn.Conv2d(conv_out_features_1, conv_out_features_2, kernel_size)
        self.fc1 = nn.Linear(
            conv_out_features_2 * cnn_final_convolution_size(input_shape[1], kernel_size, pooling_size) * cnn_final_convolution_size(input_shape[2], kernel_size, pooling_size),
            linear_size_1) 
        self.fc2 = nn.Linear(linear_size_1, linear_size_2)
        self.fc3 = nn.Linear(linear_size_2, output_shape) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

ResNet Model:

In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class MRI_ResNet(nn.Module):
    def __init__(self, block, layers):
        num_classes = 4

        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

PreTrained ResNet:

In [5]:
class MRI_ResNet2(nn.Module):
    def __init__(self):
        num_classes=4
        
        super(MRI_ResNet2, self).__init__()
        self.model = Model(weights=ModelW.IMAGENET1K_V1)
        self.model.fc = nn.Linear(in_features=self.model.fc.in_features, out_features=num_classes)

    def forward(self, x):
        return self.model(x)


AlexNet

In [6]:
class MRI_AlexNet(nn.Module):
    def __init__(self):
        num_classes = 4
        
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.fc_input_features = 256 * 7 * 7 
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self.fc_input_features, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), self.fc_input_features)
        x = self.classifier(x)
        return x


: 